In [26]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

df_distances = pd.read_csv(r'.\\FULLDATASET.csv')


"""
odd = np.arange(0,23,2)
even = np.arange(1,23,2)
X_train = df_distances.loc[odd, 'Distance (1 to 2)':'Distance (67 to 68)']
y_train = df_distances.loc[odd,'Result']

X_test = df_distances.loc[even, 'Distance (1 to 2)':'Distance (67 to 68)']
y_test = df_distances.loc[even,'Result']
"""

X = df_distances.loc[:, 'Distance (1 to 2)':'Distance (67 to 68)']
y = df_distances['Result']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=2024)


# Standardize features before PCA
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply PCA
pca = PCA()
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.fit_transform(X_test_scaled)

cl = MLPClassifier(random_state=50, max_iter=10000).fit(X_train, y_train)

print(cl.predict(X_test))
print('\n',y_test.T)
cl.score(X_test,y_test)

['L' 'L' 'L' 'L' 'L']

 37    L
16    L
41    L
39    L
24    L
Name: Result, dtype: object


1.0

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

# Train a model with one principal component
X_train_pca_single = X_train_pca[:, :1]  # Only the first component
X_test_pca_single = X_test_pca[:, :1]

# Logistic Regression
log_reg = LogisticRegression(random_state=50)
log_reg.fit(X_train, y_train)
print("Logistic Regression Accuracy:", log_reg.score(X_test, y_test))

# Decision Tree
decision_tree = DecisionTreeClassifier(random_state=50)
decision_tree.fit(X_train, y_train)
print("Decision Tree Accuracy:", decision_tree.score(X_test, y_test))

# Random forest
from sklearn.ensemble import RandomForestClassifier
# set up model: use all default values for tuning parameters
rand_for = RandomForestClassifier(random_state=2024)
# fit the model
rand_for.fit(X_train, y_train)
print("Random Forest Accuracy:", rand_for.score(X_test, y_test))





C:\Users\tomas\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression Accuracy: 1.0
Decision Tree Accuracy: 1.0
Random Forest Accuracy: 1.0
Calculated distances: [[ 52.15361924 103.3924562  154.93547044 ...  28.16025568  56.0357029
   28.44292531]]


In [79]:
import dlib
import cv2
import numpy as np
from itertools import combinations
import os

def calculate_euclidean_distances(image_path):
    """
    Calculate Euclidean distances between all pairs of facial landmarks from a given image.
    
    Parameters:
        image_path (str): Path to the image file.

    Returns:
        np.ndarray: A 2D array with shape (1, 2278) containing Euclidean distances.
    """
    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError("Image not found or unable to load. Check the image path.")

    # Initialize dlib's face detector and shape predictor
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")  # Make sure to download this file

    # Detect faces in the image
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    if len(faces) == 0:
        raise ValueError("No face detected in the image.")

    # Assume only one face and extract the first detected face
    face = faces[0]
    landmarks = predictor(gray, face)

    # Extract (x, y) coordinates of the 68 landmarks
    points = np.array([[landmarks.part(i).x, landmarks.part(i).y] for i in range(68)])

    # Plot the landmarks on the image
    for (x, y) in points:
        cv2.circle(image, (x, y), radius=2, color=(0, 255, 0), thickness=-1)  # Green dots

    # Display the image with landmarks
    cv2.imshow("Image with Landmarks", image)
    cv2.waitKey(0)  # Wait for a key press to close the window
    cv2.destroyAllWindows()

    # Calculate Euclidean distances between all pairs of landmarks
    distances = []
    for (i, j) in combinations(range(68), 2):
        dist = np.linalg.norm(points[i] - points[j])
        distances.append(dist)

    # Convert to 2D array with shape (1, 2278)
    distances_array = np.array(distances).reshape(1, -1)

    return rand_for.predict(distances_array), rand_for.predict_proba(distances_array)



image_path = os.path.join("images_comparison", "eyebrow_position_1.png")
#image_path = os.path.join("images", "1_2.jpg")
print("This person's outcome in the US Elections is", calculate_euclidean_distances(image_path = image_path))


This person's outcome in the US Elections is (array(['W'], dtype=object), array([[0.48, 0.52]]))


C:\Users\tomas\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\tomas\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
